# What's Cooking

![image](Kaggle_Project.PNG)

## Proje adımları

**Bu projede tarifleri verilen yemeklerin hangi ülkenin yemeği olduğunu tahmin eden modeli oluşturup bunu arayüze sahip bir programa yerleştirdim.**

**Veri kümesinde tarifin ID'si , yerel mutfağı, ve tariflerin olduğu sütünlar var, bü sütunlardan tariflerin olduğu sütuna doğal dil işleme methodlarını kullanacağız ve işleme alacağız.**

**Daha sonrasında ise bunu bir uygulamaya yerleştirip görsellik katacağız**

## Kurulum

In [1]:
import pandas as pd

In [2]:
df = pd.read_json(open("train.json", "r", encoding="utf8"))

In [3]:
df['ingredients'][12]

['Italian parsley leaves',
 'walnuts',
 'hot red pepper flakes',
 'extra-virgin olive oil',
 'fresh lemon juice',
 'trout fillet',
 'garlic cloves',
 'chipotle chile',
 'fine sea salt',
 'flat leaf parsley']

In [4]:
df['ingredients'].head()

0    [romaine lettuce, black olives, grape tomatoes...
1    [plain flour, ground pepper, salt, tomatoes, g...
2    [eggs, pepper, salt, mayonaise, cooking oil, g...
3                  [water, vegetable oil, wheat, salt]
4    [black pepper, shallots, cornflour, cayenne pe...
Name: ingredients, dtype: object

**Burada ise tariflerde kaç adet farklı içerik olduğunu bulmuş olduk**

In [5]:
setim = set() # Burada set kullandım çünkü listede eğer aynı veriden varsa bunlardan tekini alacak ve bize 
              # farklı olanları verecek

for i in df['ingredients']: 
    for j in i:
        setim.add(j)
len(setim)        

6714

**6714 farklı malzeme olduğunu görmüş olduk**

## Veri İşlemleri

In [6]:
def clearingandconverting(text):
    
      
    text =" ".join(text)  # Virgüllerle ayrılmış listeyi join methodu ile bir cümle haline getirdim
    
    text=text.lower()                    # Buradan sonraki 4 satırd ise NLP methodlarını uygulayabilmek adına
                                         # bütün veriyi küçük harflere çevirdik ve içlerinden numerik 
                                         # verileri ve de sembolleri attık
    text=text.replace("[^\w\s]","") 
    text=text.replace("\d+","") 
    text=text.replace("\n"," ").replace("\r","") 
    
    return text

In [7]:
df['ingredients'] = df['ingredients'].apply(clearingandconverting)

In [8]:
df['ingredients'].head(10)

0    romaine lettuce black olives grape tomatoes ga...
1    plain flour ground pepper salt tomatoes ground...
2    eggs pepper salt mayonaise cooking oil green c...
3                       water vegetable oil wheat salt
4    black pepper shallots cornflour cayenne pepper...
5    plain flour sugar butter eggs fresh ginger roo...
6    olive oil salt medium shrimp pepper garlic cho...
7    sugar pistachio nuts white almond bark flour v...
8    olive oil purple onion fresh pineapple pork po...
9    chopped tomatoes fresh basil garlic extra-virg...
Name: ingredients, dtype: object

# Modelleme

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from textblob import TextBlob
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english') 

def split_into_lemmas(text):    # Stemma analiz methodunu tanımladık
    
    text = str(text).lower()   
    
    words = TextBlob(text).words
    
    return [stemmer.stem(word) for word in words]

## Verileri Ayırma ve Vektörize Etme İşlemleri

In [10]:
x,y=df['ingredients'],df['cuisine']

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=80)

In [12]:
vect=CountVectorizer(lowercase=True, stop_words='english', ngram_range=(1,2), analyzer=split_into_lemmas)
x_train_dtm=vect.fit_transform(x_train,y_train)
x_test_dtm=vect.transform(x_test)

In [13]:
x_train_dtm

<29830x2618 sparse matrix of type '<class 'numpy.int64'>'
	with 557553 stored elements in Compressed Sparse Row format>

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [15]:
b=MultinomialNB()
model=b.fit(x_train_dtm,y_train)
b_predict=b.predict(x_test_dtm)

In [16]:
accuracy_score(y_test,b_predict)

0.7247586484312148

In [17]:
def vectorizing(text):
    
    return vect.transform([text])


In [18]:
df2 = pd.read_json(open("train.json", "r", encoding="utf8"))

In [19]:
model.predict(vectorizing(clearingandconverting(df2['ingredients'][12])))

array(['italian'], dtype='<U12')

# Test Verilerini Oluşturulan Model İle Tahmin Etme Ve Kaggle İçin Hazırlama

In [20]:
test = pd.read_json('test.json')

In [21]:
test

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."
...,...,...
9939,30246,"[large egg yolks, fresh lemon juice, sugar, bo..."
9940,36028,"[hot sauce, butter, sweet potatoes, adobo sauc..."
9941,22339,"[black pepper, salt, parmigiano reggiano chees..."
9942,42525,"[cheddar cheese, cayenne, paprika, plum tomato..."


In [22]:
test['ingredients'] = test['ingredients'].apply(clearingandconverting)

In [23]:
vext_mod = test['ingredients'].apply(vectorizing)

In [24]:
list = []

In [25]:
for v in vext_mod:
    list.append(model.predict(v))
list

[array(['british'], dtype='<U12'),
 array(['southern_us'], dtype='<U12'),
 array(['italian'], dtype='<U12'),
 array(['cajun_creole'], dtype='<U12'),
 array(['italian'], dtype='<U12'),
 array(['southern_us'], dtype='<U12'),
 array(['french'], dtype='<U12'),
 array(['chinese'], dtype='<U12'),
 array(['mexican'], dtype='<U12'),
 array(['british'], dtype='<U12'),
 array(['italian'], dtype='<U12'),
 array(['greek'], dtype='<U12'),
 array(['indian'], dtype='<U12'),
 array(['italian'], dtype='<U12'),
 array(['british'], dtype='<U12'),
 array(['french'], dtype='<U12'),
 array(['southern_us'], dtype='<U12'),
 array(['southern_us'], dtype='<U12'),
 array(['mexican'], dtype='<U12'),
 array(['southern_us'], dtype='<U12'),
 array(['japanese'], dtype='<U12'),
 array(['indian'], dtype='<U12'),
 array(['french'], dtype='<U12'),
 array(['vietnamese'], dtype='<U12'),
 array(['italian'], dtype='<U12'),
 array(['southern_us'], dtype='<U12'),
 array(['vietnamese'], dtype='<U12'),
 array(['korean'], dtype='

In [26]:
df = pd.DataFrame(list)

In [31]:
df['cuisine']=df.iloc[:,0]

In [32]:
test

,id,ingredients,cuisine
0,18009,baking powder eggs all-purpose flour raisins m...,british
1,28583,sugar egg yolks corn starch cream of tartar ba...,southern_us
2,41580,sausage links fennel bulb fronds olive oil cub...,italian
3,29752,meat cuts file powder smoked sausage okra shri...,cajun_creole
4,35687,ground black pepper salt sausage casings leeks...,italian
...,...,...,...
9939,30246,large egg yolks fresh lemon juice sugar bourbo...,french
9940,36028,hot sauce butter sweet potatoes adobo sauce salt,southern_us
9941,22339,black pepper salt parmigiano reggiano cheese r...,italian
9942,42525,cheddar cheese cayenne paprika plum tomatoes g...,cajun_creole


In [33]:
df['0'] =test['id']

In [34]:
df.columns = ['0', 'cuisine', 'id']

In [35]:
df.drop('0', axis=1, inplace=True)

In [36]:
df

,cuisine,id
0,18009,british
1,28583,southern_us
2,41580,italian
3,29752,cajun_creole
4,35687,italian
...,...,...
9939,30246,french
9940,36028,southern_us
9941,22339,italian
9942,42525,cajun_creole


In [37]:
df.set_index('id', inplace=True)

In [38]:
df

,cuisine
id,
british,18009
southern_us,28583
italian,41580
cajun_creole,29752
italian,35687
...,...
french,30246
southern_us,36028
italian,22339


In [39]:
df.to_csv('submission.csv')

# Uygulama Oluşturma

In [ ]:
import PySimpleGUI as sg
import time
from textblob import TextBlob


layout = [[sg.Text('Yemek Mutfağı Tahminleme', font=("Helvetica", 25))],
          [sg.Image(filename=r'C:\Users\MONSTERHAN\Desktop\nltk.png', size=(75, 81))],
              [sg.Text('Liste halinde malzemeleri giriniz')],
              [sg.In(justification='center', key='-IN-', enable_events=True, size=(60, 60))],
              [],
              [sg.Button('Analiz et'), sg.Text(size=(25,1), key='-OUTPUT-')],
              [sg.Button('Exit')],
              [sg.Text('Created by Erhan Namlı', font=("Helvetica", 8))]]

window = sg.Window('', layout, element_justification='c')

while True:  # Event Loop
    
    event, values = window.read()
    
    if event == sg.WIN_CLOSED or event == 'Exit':
        
        break
        
    if event == 'Analiz et':
        
        sonuc = model.predict(vectorizing(clearingandconverting(values['-IN-'])))
        
        window['-OUTPUT-'].update(sonuc)
    


![image](Program.PNG)